In [17]:
### ALL Packages Go HERE ###

import pandas as pd

In [ ]:
### Tori to FIX file setup ###

###Connection info for GitHub File###
url = 'https://github.com/TexasTori/ADS500B/raw/23fed13154371851d00cb375e177636726bf8bc2/house_sales.csv'
df_orig = pd.read_csv(url, sep=',')

###set up to automate files print location for team###
cwd = os.getcwd() 
log_prefix = os.path.join(cwd,'log_files')
print(log_prefix)

In [20]:
data = pd.read_excel('/Users/bobbymarriott/Desktop/Data Mining Class/Project.xlsx')

In [21]:
data.head(5)

,ID,ESA2008,ESA2009,ESA2010,ESA2011,ESA2012,ESA2013,ESA2014,ESA2015,ESA2016,...,Platformscount,Protagonist,MainPC,PCTYPE,CharacterPrespective,Narrative,DigitalGameMainGenre,DigitalGameSecondaryGenre,DigitalGameThirdGenre,DigitalGameGenreOTHER
0,393,0,0,0,0,0,0,0,0,0,...,6.0,Conway,Conway,3,3,1,Adventure,,,
1,390,0,0,0,0,0,0,0,0,0,...,3.0,OuterWildsTheProtagonist,OuterWildsTheProtagonist,3,1,1,ActionAdventure,,,
2,391,0,0,0,0,0,0,0,0,0,...,3.0,LeonS.Kennedy,LeonS.Kennedy,3,3,1,Survival,Horror,,
3,392,0,0,0,0,0,0,0,0,0,...,8.0,0,TheIronclad,0,4,0,Strategy,,,
4,376,0,0,0,0,0,0,0,0,0,...,3.0,KassandraorAlexios,KassandraorAlexios,3,3,1,Action,RPG,,


In [24]:
data.isnull().sum()

ID                           0
ESA2008                      0
ESA2009                      0
ESA2010                      0
ESA2011                      0
ESA2012                      0
ESA2013                      0
ESA2014                      0
ESA2015                      0
ESA2016                      0
ESA2017                      0
ESA2018                      0
ESACount                     0
EuroGamer                    0
GamesRadar                   0
Guardien                     0
IGN                          0
MetaCritic                   0
Polygon                      0
TIME                         0
CriticCount                  0
TotalCitationsCount          0
DigitalGame                  0
Year                         0
YearInterval                 0
Series                       0
Mode                         0
SpatialDimensionality        0
Platformscount               1
Protagonist                  0
MainPC                       0
PCTYPE                       0
Characte

1. Project Definition: Consider what specific metrics will define 'playability' or 'acclaim'—user ratings, hours played, critic scores, etc. Or is there a target variable that defines this for your? Your analysis will depend heavily on these definitions

Highly Playable variables: Mode (single player, multiplayer, MMO), Spatial Dimensionality (2-d, 3-d), Platform count, Character Perspective, Narrative, Digital Game Main Genre, Critic Scores 

Reflect on how the different performance metrics will guide business decisions. For example, if your model has high precision, it could be particularly useful in identifying games that are certain to receive acclaim, guiding marketing efforts to focus resources on these titles.

Conversely, a model with high recall could prevent overlooking games that could become highly playable, informing strategies to ensure broad coverage in promotional activities.

2. Data Exploration: Conduct a thorough exploratory data analysis to understand the variables at your disposal. For games, variables can be interdependent in complex ways. Player reviews might correlate with playtime, and genre might influence critical acclaim.

Patricio - EDA
Tori - Cleaning Normalizing/standarizing - to prep for models
-mod 3/4 training and testing data split.
-Feature selection

3. Modeling Pathway: Given the binary nature of your classification problem, you might start with logistic regression to establish a baseline. As you progress, explore tree-based models or random forest or even neural networks that could capture more subtle patterns within your data. 4. Model Evaluation: Alongside standard classification metrics such as accuracy, precision, recall, consider the specific nature of your project's objectives.

# Predict how many awards based off the features

## Do we need a training and a test data set?  --YES

# EDA?  Sort data at all?  Use a plot to find outliers? Histogram/Distribution of some sort?  Maybe binning of spatial dimensionality games or by narrative?  --Patricio

# Do we need to convert/standardize/normalize any of this data?   ---TORI

# C5.0 Model and then also a summary of it

# CART Model and summary

# Logistic Regression and summary

# Random Forest and summary

# Naive Bayes and summary

# ROC Curves? -maybe

## Model Evaluation Table